In [1]:
import requests
import pandas as pd
import os

In [2]:
# Create output folder
output_folder = "climate"
os.makedirs(output_folder, exist_ok=True)

In [3]:
# Read the new station_id and airport match
data = pd.read_csv('match.csv')

# Convert the data to a dictionary for easy lookup
airport_dict = dict(zip(data.iloc[:, 0], data.iloc[:, 1]))

# Define the function
def id(airport):
    return airport_dict.get(airport)

In [4]:
#retry the failed downloads with new stationid
fail_file = "fails.csv"
fails = pd.read_csv(fail_file)
#fails

### we leave PIH in 18-24 as missing data, we cannot find any climate data of them.
### MQT in 21-24, JHM in 22 and 24, FCA and PGV in24 
### ISN is closed in 2020
### We plug in BJI and EGE's data in 2024 with the 3rd closest station_id's data
### link of DHN's data link in 23 is broken

In [5]:
base_url = "https://www.ncei.noaa.gov/oa/local-climatological-data/v2/access/{}/LCD_{}_{}.csv"

retry_failed_downloads = []

# Download data
for index, row in fails.iterrows():
    airport = row["Airport"]
    year = row["Year"]
    station_id = id(airport)
        
    # Build URL and file name
    url = base_url.format(year, station_id, year)
    filename = os.path.join(output_folder, f"LCD_{airport}_{year}.csv")

    # Request and save data
    response = requests.get(url)
    if response.status_code == 200:
        with open(filename, 'wb') as file:
            file.write(response.content)
        #print(f"{airport} in {year} retry successfully")
    else:
        print(f"Failed to retry for {airport} in {year}. Status code: {response.status_code}")
        retry_failed_downloads.append({"Airport": airport, "Year": year})

Failed to retry for PIH in 2018. Status code: 404
Failed to retry for PIH in 2019. Status code: 404
Failed to retry for PIH in 2020. Status code: 404
Failed to retry for ISN in 2021. Status code: 404
Failed to retry for MQT in 2021. Status code: 404
Failed to retry for PIH in 2021. Status code: 404
Failed to retry for ISN in 2022. Status code: 404
Failed to retry for JHM in 2022. Status code: 404
Failed to retry for MQT in 2022. Status code: 404
Failed to retry for PIH in 2022. Status code: 404
Failed to retry for DHN in 2023. Status code: 502
Failed to retry for ISN in 2023. Status code: 404
Failed to retry for MQT in 2023. Status code: 404
Failed to retry for PIH in 2023. Status code: 404
Failed to retry for FCA in 2024. Status code: 404
Failed to retry for ISN in 2024. Status code: 404
Failed to retry for JHM in 2024. Status code: 404
Failed to retry for MQT in 2024. Status code: 404
Failed to retry for PGV in 2024. Status code: 404
Failed to retry for PIH in 2024. Status code: 404
